<a href="https://colab.research.google.com/github/Gerda-T/Energy_Bench_Marking/blob/main/Building_energy_benchmarking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression, ridge_regression, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
from google.colab import files
uploaded = files.upload()

Saving 2016_Building_Energy_Benchmarking.csv to 2016_Building_Energy_Benchmarking.csv


In [4]:
df = pd.read_csv("2016_Building_Energy_Benchmarking.csv")
df.head()

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
0,1,2016,NonResidential,Hotel,Mayflower park hotel,405 Olive way,Seattle,WA,98101.0,0659000030,7,DOWNTOWN,47.61220,-122.33799,1927,1.0,12,88434,0,88434,Hotel,Hotel,88434.0,NaN,NaN,NaN,NaN,NaN,60.0,81.699997,84.300003,182.500000,189.000000,7226362.5,7456910.0,2003882.00,1.156514e+06,3946027.0,12764.52930,1276453.0,False,NaN,Compliant,NaN,249.98,2.83
1,2,2016,NonResidential,Hotel,Paramount Hotel,724 Pine street,Seattle,WA,98101.0,0659000220,7,DOWNTOWN,47.61317,-122.33393,1996,1.0,11,103566,15064,88502,"Hotel, Parking, Restaurant",Hotel,83880.0,Parking,15064.0,Restaurant,4622.0,NaN,61.0,94.800003,97.900002,176.100006,179.399994,8387933.0,8664479.0,0.00,9.504252e+05,3242851.0,51450.81641,5145082.0,False,NaN,Compliant,NaN,295.86,2.86
2,3,2016,NonResidential,Hotel,5673-The Westin Seattle,1900 5th Avenue,Seattle,WA,98101.0,0659000475,7,DOWNTOWN,47.61393,-122.33810,1969,1.0,41,956110,196718,759392,Hotel,Hotel,756493.0,NaN,NaN,NaN,NaN,NaN,43.0,96.000000,97.699997,241.899994,244.100006,72587024.0,73937112.0,21566554.00,1.451544e+07,49526664.0,14938.00000,1493800.0,False,NaN,Compliant,NaN,2089.28,2.19
3,5,2016,NonResidential,Hotel,HOTEL MAX,620 STEWART ST,Seattle,WA,98101.0,0659000640,7,DOWNTOWN,47.61412,-122.33664,1926,1.0,10,61320,0,61320,Hotel,Hotel,61320.0,NaN,NaN,NaN,NaN,NaN,56.0,110.800003,113.300003,216.199997,224.000000,6794584.0,6946800.5,2214446.25,8.115253e+05,2768924.0,18112.13086,1811213.0,False,NaN,Compliant,NaN,286.43,4.67
4,8,2016,NonResidential,Hotel,WARWICK SEATTLE HOTEL (ID8),401 LENORA ST,Seattle,WA,98121.0,0659000970,7,DOWNTOWN,47.61375,-122.34047,1980,1.0,18,175580,62000,113580,"Hotel, Parking, Swimming Pool",Hotel,123445.0,Parking,68009.0,Swimming Pool,0.0,NaN,75.0,114.800003,118.699997,211.399994,215.600006,14172606.0,14656503.0,0.00,1.573449e+06,5368607.0,88039.98438,8803998.0,False,NaN,Compliant,NaN,505.01,2.88


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3376 entries, 0 to 3375
Data columns (total 46 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   OSEBuildingID                    3376 non-null   int64  
 1   DataYear                         3376 non-null   int64  
 2   BuildingType                     3376 non-null   object 
 3   PrimaryPropertyType              3376 non-null   object 
 4   PropertyName                     3376 non-null   object 
 5   Address                          3376 non-null   object 
 6   City                             3376 non-null   object 
 7   State                            3376 non-null   object 
 8   ZipCode                          3360 non-null   float64
 9   TaxParcelIdentificationNumber    3376 non-null   object 
 10  CouncilDistrictCode              3376 non-null   int64  
 11  Neighborhood                     3376 non-null   object 
 12  Latitude            

In [6]:
df.describe()

,OSEBuildingID,DataYear,ZipCode,CouncilDistrictCode,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),LargestPropertyUseTypeGFA,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseTypeGFA,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),Comments,TotalGHGEmissions,GHGEmissionsIntensity
count,3376.000000,3376.0,3360.000000,3376.000000,3376.000000,3376.000000,3376.000000,3368.000000,3376.000000,3.376000e+03,3376.000000,3.376000e+03,3.356000e+03,1679.000000,596.000000,2533.000000,3369.000000,3370.000000,3367.000000,3367.000000,3.371000e+03,3.370000e+03,3.367000e+03,3.367000e+03,3.367000e+03,3.367000e+03,3.367000e+03,0.0,3367.000000,3367.000000
mean,21208.991114,2016.0,98116.949107,4.439277,47.624033,-122.334795,1968.573164,1.106888,4.709123,9.483354e+04,8001.526066,8.683201e+04,7.917764e+04,28444.075817,11738.675166,67.918674,54.732116,57.033798,134.232848,137.783932,5.403667e+06,5.276726e+06,2.745959e+05,1.086639e+06,3.707612e+06,1.368505e+04,1.368505e+06,NaN,119.723971,1.175916
std,12223.757015,0.0,18.615205,2.120625,0.047758,0.027203,33.088156,2.108402,5.494465,2.188376e+05,32326.723928,2.079398e+05,2.017034e+05,54392.917928,29331.199286,26.873271,56.273124,57.163330,139.287554,139.109807,2.161063e+07,1.593879e+07,3.912173e+06,4.352478e+06,1.485066e+07,6.709781e+04,6.709781e+06,NaN,538.832227,1.821452
min,1.000000,2016.0,98006.000000,1.000000,47.499170,-122.414250,1900.000000,0.000000,0.000000,1.128500e+04,0.000000,3.636000e+03,5.656000e+03,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,-2.100000,0.000000e+00,0.000000e+00,0.000000e+00,-3.382680e+04,-1.154170e+05,0.000000e+00,0.000000e+00,NaN,-0.800000,-0.020000
25%,19990.750000,2016.0,98105.000000,3.000000,47.599860,-122.350662,1948.000000,1.000000,2.000000,2.848700e+04,0.000000,2.775600e+04,2.509475e+04,5000.000000,2239.000000,53.000000,27.900000,29.400000,74.699997,78.400002,9.251286e+05,9.701822e+05,0.000000e+00,1.874229e+05,6.394870e+05,0.000000e+00,0.000000e+00,NaN,9.495000,0.210000
50%,23112.000000,2016.0,98115.000000,4.000000,47.618675,-122.332495,1975.000000,1.000000,4.000000,4.417500e+04,0.000000,4.321600e+04,3.989400e+04,10664.000000,5043.000000,75.000000,38.599998,40.900002,96.199997,101.099998,1.803753e+06,1.904452e+06,0.000000e+00,3.451299e+05,1.177583e+06,3.237538e+03,3.237540e+05,NaN,33.920000,0.610000
75%,25994.250000,2016.0,98122.000000,7.000000,47.657115,-122.319407,1997.000000,1.000000,5.000000,9.099200e+04,0.000000,8.427625e+04,7.620025e+04,26640.000000,10138.750000,90.000000,60.400002,64.275002,143.899994,148.349998,4.222455e+06,4.381429e+06,0.000000e+00,8.293178e+05,2.829632e+06,1.189033e+04,1.189034e+06,NaN,93.940000,1.370000
max,50226.000000,2016.0,98272.000000,7.000000,47.733870,-122.220966,2015.000000,111.000000,99.000000,9.320156e+06,512608.000000,9.320156e+06,9.320156e+06,686750.000000,459748.000000,100.000000,834.400024,834.400024,2620.000000,2620.000000,8.739237e+08,4.716139e+08,1.349435e+08,1.925775e+08,6.570744e+08,2.979090e+06,2.979090e+08,NaN,16870.980000,34.090000


In [7]:
df.isna().sum()

OSEBuildingID                         0
DataYear                              0
BuildingType                          0
PrimaryPropertyType                   0
PropertyName                          0
Address                               0
City                                  0
State                                 0
ZipCode                              16
TaxParcelIdentificationNumber         0
CouncilDistrictCode                   0
Neighborhood                          0
Latitude                              0
Longitude                             0
YearBuilt                             0
NumberofBuildings                     8
NumberofFloors                        0
PropertyGFATotal                      0
PropertyGFAParking                    0
PropertyGFABuilding(s)                0
ListOfAllPropertyUseTypes             9
LargestPropertyUseType               20
LargestPropertyUseTypeGFA            20
SecondLargestPropertyUseType       1697
SecondLargestPropertyUseTypeGFA    1697





Do all the transformations necessary: Standardization, use cross validation and grid search to select the best hyperparameters

Try different regression models

Model evaluation with metrics



In [8]:
df.shape

(3376, 46)

Exploratory Data Analysis

In [9]:
df['SecondLargestPropertyUseType'].unique()

array([nan, 'Parking', 'Office', 'Restaurant', 'K-12 School',
       'Laboratory', 'Refrigerated Warehouse',
       'Non-Refrigerated Warehouse', 'Other - Education',
       'Vocational School', 'Retail Store',
       'Personal Services (Health/Beauty, Dry Cleaning, etc)',
       'Other - Entertainment/Public Assembly', 'Data Center',
       'Swimming Pool', 'Other', 'Courthouse', 'Residence Hall/Dormitory',
       'Distribution Center', 'Self-Storage Facility', 'Other - Services',
       'Movie Theater', 'Supermarket/Grocery Store', 'Medical Office',
       'Other - Recreation', 'Convenience Store without Gas Station',
       'Performing Arts', 'College/University', 'Other - Restaurant/Bar',
       'Adult Education', 'Enclosed Mall', 'Multifamily Housing',
       'Bar/Nightclub', 'Financial Office',
       'Repair Services (Vehicle, Shoe, Locksmith, etc)',
       'Manufacturing/Industrial Plant', 'Other - Lodging/Residential',
       'Bank Branch', 'Food Service', 'Worship Facility',


In [10]:
fig = px.bar(df, x='ThirdLargestPropertyUseType', y='Electricity(kWh)',
             title='Energy Consumption by Third Largest Property Use Type',
             labels={'ThirdLargestPropertyUseType': 'Third Largest Property Use Type',
                     'EnergyConsumption': 'Energy Consumption'},
             height=1000, width=1000)

fig.update_layout(xaxis=dict(tickangle=45))

fig.show()


In [11]:
fig = px.bar(df, x='SecondLargestPropertyUseType', y='Electricity(kWh)',
             title='Energy Consumption by Second Largest Property Use Type',
             labels={'SecondLargestPropertyUseType': 'Second Largest Property Use Type',
                     'EnergyConsumption': 'Energy Consumption'},
             height=1000, width=1000)

fig.update_layout(xaxis=dict(tickangle=45))

fig.show()


In [ ]:
df['PrimaryPropertyType'].unique()

array(['Hotel', 'Other', 'Mid-Rise Multifamily', 'Mixed Use Property',
       'K-12 School', 'University', 'Small- and Mid-Sized Office',
       'Self-Storage Facility', 'Warehouse', 'Large Office',
       'Senior Care Community', 'Medical Office', 'Retail Store',
       'Hospital', 'Residence Hall', 'Distribution Center',
       'Worship Facility', 'Low-Rise Multifamily',
       'Supermarket / Grocery Store', 'Laboratory',
       'Refrigerated Warehouse', 'Restaurant', 'High-Rise Multifamily',
       'Office'], dtype=object)

In [ ]:
fig = px.bar(df, x='PrimaryPropertyType', y='Electricity(kWh)',
             title='Energy Consumption by Primary Property Type',
             labels={'PrimaryPropertyType': 'Primary Property Type',
                     'EnergyConsumption': 'Energy Consumption'},
             height=1000, width=1000)

fig.update_layout(xaxis=dict(tickangle=45))

fig.show()


In [12]:
df['BuildingType'].unique()

array(['NonResidential', 'Nonresidential COS', 'Multifamily MR (5-9)',
       'SPS-District K-12', 'Campus', 'Multifamily LR (1-4)',
       'Multifamily HR (10+)', 'Nonresidential WA'], dtype=object)

In [ ]:
fig = px.bar(df, x='BuildingType', y='Electricity(kWh)',
             title='Energy Consumption by Building Type',
             labels={'BuildingType': 'Building Type',
                     'Electricity': 'Energy Consumption'},
             height=1000, width=1000)

fig.update_layout(xaxis=dict(tickangle=45))

fig.show()


Note that the target is TotalGHGEmissions and SiteEnergyUse(kBtu) for non residential buildings


In [13]:
df['BuildingType'].unique()

array(['NonResidential', 'Nonresidential COS', 'Multifamily MR (5-9)',
       'SPS-District K-12', 'Campus', 'Multifamily LR (1-4)',
       'Multifamily HR (10+)', 'Nonresidential WA'], dtype=object)

Data Pre-Processing

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3376 entries, 0 to 3375
Data columns (total 46 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   OSEBuildingID                    3376 non-null   int64  
 1   DataYear                         3376 non-null   int64  
 2   BuildingType                     3376 non-null   object 
 3   PrimaryPropertyType              3376 non-null   object 
 4   PropertyName                     3376 non-null   object 
 5   Address                          3376 non-null   object 
 6   City                             3376 non-null   object 
 7   State                            3376 non-null   object 
 8   ZipCode                          3360 non-null   float64
 9   TaxParcelIdentificationNumber    3376 non-null   object 
 10  CouncilDistrictCode              3376 non-null   int64  
 11  Neighborhood                     3376 non-null   object 
 12  Latitude            

In [18]:
new_df = (
    df
   .drop(['Comments',
          'Outlier',
          'YearsENERGYSTARCertified',
          'DataYear',
          'OSEBuildingID',
          'City',
          'State',
          'DefaultData',
          'ComplianceStatus'], axis=1)
   .query("BuildingType == 'NonResidential' | BuildingType == 'Nonresidential COS' | BuildingType == 'Campus' | BuildingType == 'Nonresidential WA'")
   .assign(
        ThirdLargestPropertyUseType=df['ThirdLargestPropertyUseType'].fillna('None'),
        ThirdLargestPropertyUseTypeGFA=df['ThirdLargestPropertyUseTypeGFA'].fillna(0),
        ENERGYSTARScore=df['ENERGYSTARScore'].fillna(0)
    )
   .dropna(subset=['NumberofBuildings',
                   'ZipCode',
                   'SecondLargestPropertyUseTypeGFA',
                   'GHGEmissionsIntensity',
                   'SteamUse(kBtu)',
                   'SiteEnergyUseWN(kBtu)',
                   'SiteEUIWN(kBtu/sf)',
                   'SiteEUI(kBtu/sf)'
                   ])
   .astype({'NumberofBuildings': 'int',
            'ENERGYSTARScore' : 'int',
            'SecondLargestPropertyUseTypeGFA': 'int'})
   .assign(
        ZipCode=lambda x: x['ZipCode'].astype('Int64').astype(str),
        TaxParcelIdentificationNumber=lambda x: x['TaxParcelIdentificationNumber'].str.extract('(\d+)').astype(float).astype('Int64')
    )
   .rename(columns = {"PropertyGFABuilding(s)" : "PropertyGFABuildings",
                      "SiteEUI(kBtu/sf)" : "SiteEUI_kBtu_sf",
                      "SiteEUIWN(kBtu/sf)" : "SiteEUIWN_kBtu_sf",
                      "SourceEUI(kBtu/sf)" : "SourceEUI_kBtu_sf",
                      "SourceEUIWN(kBtu/sf)" : "SourceEUIWN_kBtu_sf",
                      "SiteEnergyUse(kBtu)" : "SiteEnergyUse_kBtu",
                      "SiteEnergyUseWN(kBtu)" : "SiteEnergyUseWN_kBtu",
                      "SteamUse(kBtu)" : "SteamUse_kBtu",
                      "Electricity(kWh)" : "Electricity_kWh",
                      "Electricity(kBtu)" : "Electricity_kBtu",
                      "NaturalGas(therms)" : "NaturalGas_therms",
                      "NaturalGas(kBtu)" : "NaturalGas_kBtu"

   })
   .pipe(lambda x: pd.get_dummies(x, columns=['Neighborhood',
                                              'BuildingType',
                                              'PrimaryPropertyType',
                                              'PropertyName',
                                              'Address',
                                              'ListOfAllPropertyUseTypes',
                                              'LargestPropertyUseType',
                                              'SecondLargestPropertyUseType',
                                              'ThirdLargestPropertyUseType',
                                              'ListOfAllPropertyUseTypes'
                                              ]))
   .assign(
        NumberofBuildings=lambda x: StandardScaler().fit_transform(x[['NumberofBuildings']]),
        CouncilDistrictCode=lambda x: StandardScaler().fit_transform(x[['CouncilDistrictCode']]),
        SecondLargestPropertyUseTypeGFA=lambda x: StandardScaler().fit_transform(x[['SecondLargestPropertyUseTypeGFA']]),
        ThirdLargestPropertyUseTypeGFA=lambda x: StandardScaler().fit_transform(x[['ThirdLargestPropertyUseTypeGFA']]),
        ENERGYSTARScore=lambda x: StandardScaler().fit_transform(x[['ENERGYSTARScore']]),
        TaxParcelIdentificationNumber=lambda x: StandardScaler().fit_transform(x[['TaxParcelIdentificationNumber']]),
        ZipCode=lambda x: StandardScaler().fit_transform(x[['ZipCode']]),
        Latitude=lambda x: StandardScaler().fit_transform(x[['Latitude']]),
        Longitude=lambda x: StandardScaler().fit_transform(x[['Longitude']]),
        YearBuilt=lambda x: StandardScaler().fit_transform(x[['YearBuilt']]),
        NumberofFloors=lambda x: StandardScaler().fit_transform(x[['NumberofFloors']]),
        PropertyGFATotal=lambda x: StandardScaler().fit_transform(x[['PropertyGFATotal']]),
        PropertyGFAParking=lambda x: StandardScaler().fit_transform(x[['PropertyGFAParking']]),
        LargestPropertyUseTypeGFA=lambda x: StandardScaler().fit_transform(x[['LargestPropertyUseTypeGFA']]),
        PropertyGFABuildings=lambda x: StandardScaler().fit_transform(x[['PropertyGFABuildings']]),
        SiteEUI_kBtu_sf=lambda x: StandardScaler().fit_transform(x[['SiteEUI_kBtu_sf']]),
        SiteEUIWN_kBtu_sf=lambda x: StandardScaler().fit_transform(x[['SiteEUIWN_kBtu_sf']]),
        SourceEUI_kBtu_sf=lambda x: StandardScaler().fit_transform(x[['SourceEUI_kBtu_sf']]),
        SourceEUIWN_kBtu_sf=lambda x: StandardScaler().fit_transform(x[['SourceEUIWN_kBtu_sf']]),
        SiteEnergyUse_kBtu=lambda x: StandardScaler().fit_transform(x[['SiteEnergyUse_kBtu']]),
        SiteEnergyUseWN_kBtu = lambda x: StandardScaler().fit_transform(x[['SiteEnergyUseWN_kBtu']]),
        SteamUse_kBtu=lambda x: StandardScaler().fit_transform(x[['SteamUse_kBtu']]),
        Electricity_kWh=lambda x: StandardScaler().fit_transform(x[['Electricity_kWh']]),
        Electricity_kBtu=lambda x: StandardScaler().fit_transform(x[['Electricity_kBtu']]),
        NaturalGas_therms=lambda x: StandardScaler().fit_transform(x[['NaturalGas_therms']]),
        NaturalGas_kBtu=lambda x: StandardScaler().fit_transform(x[['NaturalGas_kBtu']]),
        TotalGHGEmissions = lambda x: StandardScaler().fit_transform(x[['TotalGHGEmissions']]),
        GHGEmissionsIntensity = lambda x: StandardScaler().fit_transform(x[['GHGEmissionsIntensity']])


        )

   #.head()
   #.info()
   #.isna().sum()

)

i will fit transform everything at once after separating the data into train and test. look at chatjpt.

In [19]:
new_df.head()

ZipCode  TaxParcelIdentificationNumber  CouncilDistrictCode  Latitude  \
1  -0.813260                      -1.197898              1.06795 -0.034528   
4   0.322398                      -1.197898              1.06795 -0.020721   
13 -0.813260                      -1.097378              1.06795 -0.155455   
14 -0.813260                      -1.097378              1.06795 -0.168309   
15  2.196234                      -1.097378              1.06795 -0.193542   

    Longitude  YearBuilt  NumberofBuildings  NumberofFloors  PropertyGFATotal  \
1    0.000765   0.936512          -0.080718        0.726073         -0.186674   
4   -0.300469   0.472874          -0.080718        1.614564          0.107043   
13   0.084134   0.472874          -0.080718        3.518475          0.679564   
14   0.069395  -1.207815          -0.080718        0.726073         -0.233072   
15   0.044983   1.168332          -0.080718        0.726073          1.071306   

    PropertyGFAParking  PropertyGFABuildings  LargestPropertyUseTypeGFA  \
1            -0.127298             -0.178832                  -0.137296   
4             0.724604             -0.063113                   0.064635   
13            0.644743              0.604920                   0.942825   
14            0.056673             -0.278097                  -0.221458   
15            0.633853              1.050891                   1.297038   

    SecondLargestPropertyUseTypeGFA  ThirdLargestPropertyUseTypeGFA  \
1                         -0.319905                       -0.062482   
4                          0.476000                       -0.248623   
13                         0.319525                       -0.248623   
14                        -0.167534                       -0.248623   
15                         0.190244                       -0.205370   

    ENERGYSTARScore  SiteEUI_kBtu_sf  SiteEUIWN_kBtu_sf  SourceEUI_kBtu_sf  \
1          0.557996         0.191139           0.197901          -0.126518   
4          0.915724         0.450916           0.464540           0.054345   
13         0.481340        -0.059545          -0.080277          -0.107048   
14        -0.106356         1.031516           1.069608           0.494976   
15        -1.000675        -0.380370          -0.402039          -0.231552   

    SourceEUIWN_kBtu_sf  SiteEnergyUse_kBtu  SiteEnergyUseWN_kBtu  \
1             -0.121459           -0.075809             -0.070244   
4              0.063819            0.139426              0.147278   
13            -0.114805            0.442771              0.432917   
14             0.515753            0.012079              0.021045   
15            -0.241224            0.303744              0.294315   

    SteamUse_kBtu  Electricity_kWh  Electricity_kBtu  NaturalGas_therms  \
1       -0.090997        -0.254706         -0.254706           0.246210   
4       -0.090997        -0.128979         -0.128979           0.559011   
13       0.577643         0.375857          0.375857           0.155450   
14       0.659819        -0.219839         -0.219839          -0.018664   
15       0.022694         0.609309          0.609309          -0.193645   

    NaturalGas_kBtu  TotalGHGEmissions  GHGEmissionsIntensity  \
1          0.246210           0.087672               0.482352   
4          0.559011           0.324687               0.490222   
13         0.155450           0.487431               0.163591   
14        -0.018664           0.331917               1.540953   
15        -0.193645          -0.042061              -0.469996   

    Neighborhood_BALLARD  Neighborhood_Ballard  Neighborhood_CENTRAL  \
1                      0                     0                     0   
4                      0                     0                     0   
13                     0                     0                     0   
14                     0                     0                     0   
15                     0                     0                     0   

    Neighbor

In [ ]:
fig = px.bar(new_df, x='Address', y='TotalGHGEmissions',
             title='Address by Total Gas Emissions',
             labels={'Address': 'Address',
                     'TotalGHGEmissions': 'Total Gas Emissions'},
             height=1000, width=1000)

fig.update_layout(xaxis=dict(tickangle=45))

fig.show()


In [ ]:
fig = px.bar(new_df, x='Address', y='SiteEnergyUse(kBtu)',
             title='Address by Total Gas Emissions',
             labels={'Address': 'Address',
                     'SiteEnergyUse(kBtu)': 'Energy Use'},
             height=1000, width=1000)

fig.update_layout(xaxis=dict(tickangle=45))

fig.show()


Train Test Split

In [23]:
X = new_df.drop(['TotalGHGEmissions', 'SiteEnergyUse_kBtu'], axis=1)
y1 = new_df['TotalGHGEmissions']
y2 = new_df['SiteEnergyUse_kBtu']

In [24]:
X_train, X_test, y1_train, y1_test, y2_train, y2_test = train_test_split(X, y1, y2, test_size=0.2, random_state=42)

GridSearch CV

In [30]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

Grid Search on TotalGHGEmissions

In [31]:
rf_model_ghg = RandomForestRegressor(random_state=42)
grid_search_ghg = GridSearchCV(estimator=rf_model_ghg, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_ghg.fit(X_train, y1_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='neg_mean_squared_error')

In [32]:
best_params_ghg = grid_search_ghg.best_params_

In [39]:
best_rf_model_ghg = RandomForestRegressor(random_state=42, **best_params_ghg)
best_rf_model_ghg.fit(X_train, y2_train)

RandomForestRegressor(n_estimators=50, random_state=42)

 Grid search for SiteEnergyUse(kBtu)

In [33]:
rf_model_energy = RandomForestRegressor(random_state=42)
grid_search_energy = GridSearchCV(estimator=rf_model_energy, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_energy.fit(X_train, y2_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='neg_mean_squared_error')

In [34]:
best_params_energy = grid_search_energy.best_params_

In [38]:
best_rf_model_energy = RandomForestRegressor(random_state=42, **best_params_energy)
best_rf_model_energy.fit(X_train, y2_train)

RandomForestRegressor(min_samples_leaf=2, n_estimators=200, random_state=42)

In [ ]:
feature_importances_energy = best_rf_model_energy.feature_importances_
indices_energy = feature_importances_energy.argsort()[::-1]

Evaluate the models using Cross-Validation

In [40]:
cv_scores_ghg = cross_val_score(best_rf_model_ghg, X_train, y1_train, cv=5, scoring='neg_mean_squared_error')
cv_scores_energy = cross_val_score(best_rf_model_energy, X_train, y2_train, cv=5, scoring='neg_mean_squared_error')

In [41]:
print(f'Cross-Validation Scores for TotalGHGEmissions: {cv_scores_ghg}')
print(f'Mean Cross-Validation Score for TotalGHGEmissions: {cv_scores_ghg.mean()}')

print(f'Cross-Validation Scores for SiteEnergyUse(kBtu): {cv_scores_energy}')
print(f'Mean Cross-Validation Score for SiteEnergyUse(kBtu): {cv_scores_energy.mean()}')

Cross-Validation Scores for TotalGHGEmissions: [-0.0064341  -1.95154474 -0.00484374 -0.03387078 -0.04696623]
Mean Cross-Validation Score for TotalGHGEmissions: -0.40873191672694525
Cross-Validation Scores for SiteEnergyUse(kBtu): [-0.01166521 -0.88015133 -0.13951385 -0.00088752 -0.00359273]
Mean Cross-Validation Score for SiteEnergyUse(kBtu): -0.20716212680916515
